# OpenEye (OE) docking example

This notebook shows how to process data from [KLIFS](https://klifs.vu-compmedchem.nl/), perform docking into a kinase with different docking methods available in the [OpenEye toolkit](https://docs.eyesopen.com/toolkits/python/index.html) as well as how to assess and visualize docking results.

The main aim of the following sections is to get to know two docking methods (Hybrid, ChemGauss) from OEDocking. The [Hybrid](https://docs.eyesopen.com/toolkits/python/dockingtk/docking.html#hybrid-method) method uses a shape overlay to the co-crystalized ligand to bias the docking algorithm. However, it is not usable if a co-crystalized ligand is not available. In this situation, the more 'traditional' [ChemGauss](https://docs.eyesopen.com/toolkits/python/dockingtk/scoring.html#chemgauss3) docking method can be used to dock a ligand.

Re-docking will be performed to assess the performance of the two docking methods in placing the co-crystalized ligand in the corresponding protein structure. Cross-docking will be performed to assess the performance of the Hybrid method in placing a ligand into another structure of the same kinase and conformation. Here it would be interesting to evaluate, if the Hybrid method fails if the docked ligand is too dissimilar to the co-crystalized ligand used during shape overlay.

Docking results will be visualized using NGLView and assessed by calculating the RMSD between docking pose and crystal structure pose.

## Content

1. Process data from KLIFS.  
2. Re-Docking.  
   2.1. Prepare protein and ligand.  
   2.2. Hybrid docking method.  
   2.3. ChemGauss docking method.  
3. Cross-Docking.  
   3.1. Hybrid docking method.

## 1. Process data from KLIFS.

Let's retrieve all relevant data from KLIFS and store it in a single dataframe for subsequent processing.

In [1]:
import klifs_utils
kinase_ids = klifs_utils.remote.kinases.kinase_names().kinase_ID.to_list()
kinase_df = klifs_utils.remote.structures.structures_from_kinase_id(kinase_ids)
print('Number of PDB entries:', len(set(kinase_df['pdb'])))
print('Number of KLIFS entries:', len(kinase_df))
kinase_df.tail()

Number of PDB entries: 5130
Number of KLIFS entries: 11056


,structure_ID,kinase,species,kinase_ID,pdb,alt,chain,rmsd1,rmsd2,pocket,...,bp_I_A,bp_I_B,bp_II_in,bp_II_A_in,bp_II_B_in,bp_II_out,bp_II_B,bp_III,bp_IV,bp_V
11051,9098,PI4K2B,Human,1091,4wtv,B,A,1.554,3.088,ERISQGSSGSYFVGVFKPKGYLSEAGAYLVDNSIVPKTKVVGSFQL...,...,False,False,False,False,False,False,False,False,False,False
11052,9096,PI4K2B,Human,1091,4wtv,B,B,1.554,3.041,ERIS___SGSYFVGVFKPKGYLSEAGAYLVDNSIVPKTKVVGSFQL...,...,False,False,False,False,False,False,False,False,False,False
11053,9095,PI4K2B,Human,1091,4wtv,A,A,1.554,3.088,ERISQGSSGSYFVGVFKPKGYLSEAGAYLVDNSIVPKTKVVGSFQL...,...,False,False,False,False,False,False,False,False,False,False
11054,9070,PI4KA,Human,1096,6bq1,,A,1.704,2.676,_PMQSAAKAPYLAAIFKVGDCRQDMLALQIIDLFVFPYRVVCGVIE...,...,False,True,False,False,False,False,False,False,False,False
11055,9069,PI4KA,Human,1096,6bq1,,E,1.699,2.670,_PMQSAAKAPYLAAIFKVGDCRQDMLALQIIDLFVFPYRVVCGVIE...,...,True,True,False,False,False,False,False,False,False,False


Interestingly, we find several entries for the same PDB ID. Why is that?

In [2]:
kinase_df.columns

Index(['structure_ID', 'kinase', 'species', 'kinase_ID', 'pdb', 'alt', 'chain',
       'rmsd1', 'rmsd2', 'pocket', 'resolution', 'quality_score',
       'missing_residues', 'missing_atoms', 'ligand', 'allosteric_ligand',
       'DFG', 'aC_helix', 'Grich_distance', 'Grich_angle', 'Grich_rotation',
       'front', 'gate', 'back', 'fp_I', 'fp_II', 'bp_I_A', 'bp_I_B',
       'bp_II_in', 'bp_II_A_in', 'bp_II_B_in', 'bp_II_out', 'bp_II_B',
       'bp_III', 'bp_IV', 'bp_V'],
      dtype='object')

**alt** - Structures can contain alternate locations of atoms.  
**chain** - Structures can contain several chains with different geometry.  
**ligand**, **allosteric_ligand** - Structures can contain multiple ligands either in the same binding site or in different chains. In case of 6q0t there are two ligands interacting with each other, docking these ligands seperately might be challenging. In case of 1h00 the racemic mixture was crystalized revealing both enantiomers in the binding pocket, which would be suitable for docking.

To make things easier, let's pick a single entry per pdb code. Such entry should contain a single orthosteric **ligand** and should have the highest **quality_score** of all available **alt/chain combinations**.

In [3]:
# filter for orthosteric ligand
kinase_df = kinase_df[kinase_df['ligand'] != 0]
print('Number of PDB entries:', len(set(kinase_df['pdb'])))
print('Number of KLIFS entries:', len(kinase_df))

Number of PDB entries: 4501
Number of KLIFS entries: 9380


In [4]:
# filter for entries with a single orthosteric ligand
kinase_df = kinase_df.groupby('pdb').filter(lambda x: len(set(x['ligand'])) == 1)
print('Number of PDB entries:', len(set(kinase_df['pdb'])))
print('Number of KLIFS entries:', len(kinase_df))

Number of PDB entries: 4476
Number of KLIFS entries: 9271


In [5]:
# sort by alt, chain and quality score to pick representative structure in next step
kinase_df = kinase_df.sort_values(by=['alt','chain', 'quality_score'],ascending=[True, True, False])
# keep entry with highest quality score (alt 'A' preferred over alt 'B', chain 'A' preferred over 'B')
kinase_df = kinase_df.groupby('pdb').head(1)
print('Number of PDB entries:', len(set(kinase_df['pdb'])))
print('Number of KLIFS entries:', len(kinase_df))

Number of PDB entries: 4476
Number of KLIFS entries: 4476


This data set contains bullet proof cases, that an ideal docking algorithm should be able to re-dock. Let's save it for later use.

In [6]:
kinase_df.to_csv('data/re_docking_data.csv')

Another challenge for a docking program would be cross-docking, in which a co-crystalized ligand is docked into another protein structure corresponding to the same kinase. Since kinases are highly dynamic, cross-docking should only be performed within the same kinase conformations. In KLIFS unqiue kinases can be identified using **kinase_ID**. The conformation can be determined with the **DFG** and **aC_helix** fields.

In [7]:
# find structures, for which multiple entries per kinase and conformation are available
multiple_pdbs_df = kinase_df.groupby(['kinase_ID', 'DFG', 'aC_helix']).filter(lambda x: len(x[['kinase_ID', 'DFG', 'aC_helix']]) > 1)

In [8]:
# Check if command works
kinase_dict = {}
for index, row in multiple_pdbs_df.iterrows():
    identity = str(row['kinase_ID']) + row['DFG'] + row['aC_helix']
    if identity in kinase_dict.keys():
        kinase_dict[identity].append(index)
    else:
        kinase_dict[identity] = [index]
print(f'Minimum number of entries for each kinase and conformation: {min([len(x) for x in kinase_dict.values()])}')
print('Number of PDB entries:', len(set(multiple_pdbs_df['pdb'])))
print('Number of KLIFS entries:', len(multiple_pdbs_df))

Minimum number of entries for each kinase and conformation: 2
Number of PDB entries: 4278
Number of KLIFS entries: 4278


Let's save this data set for later use.

In [9]:
multiple_pdbs_df.to_csv('data/cross_docking_data.csv')

## 2. Re-Docking.

### 2.1. Prepare protein and ligand.

In this step a structure will be retrieved from PDB, and protein and ligand prepared for docking.

In [10]:
# pick structure
pdb_id = '1jvp'
chain_id, altloc, ligand_id  = kinase_df[kinase_df['pdb'] == pdb_id][['chain', 'alt', 'ligand']].values[0]
print(f'pdb: {pdb_id}\nchain: {chain_id}\naltloc: {altloc}\nligand: {ligand_id}')

pdb: 1jvp
chain: P
altloc: 1
ligand: LIG


In [11]:
# retrieve structure
from dockin.oe_docking import get_structure_from_pdb
mol = get_structure_from_pdb(pdb_id)
mol.NumAtoms()

2709

In [12]:
# select chain, alternate location and ligand
from dockin.oe_docking import select_chain, select_altloc, select_ligand
mol = select_chain(mol, chain_id)
mol = select_altloc(mol, altloc)
mol = select_ligand(mol, ligand_id)
mol.NumAtoms()

2547

In [13]:
# prepare protein and ligand
from dockin.oe_docking import prepare_complex
protein, ligand = prepare_complex(mol)
print(f'Protein: {protein.NumAtoms()}\nLigand: {ligand.NumAtoms()}')

Re-optimizing hydrogen positions...
Identifying design units...
Protein: 4631
Ligand: 29


In [14]:
# create receptor
from dockin.oe_docking import create_complex_receptor
complex_receptor = create_complex_receptor(protein, ligand)

### 2.2. Hybrid docking method.

Now we have everything to perform a Re-Docking using the Hybrid method.

In [15]:
from dockin.oe_docking import hybrid_docking
# the just extracted ligand will be passed as list, since the hybrid_docking function expects a list of molecules
docking_poses = hybrid_docking(complex_receptor, [ligand])
docking_poses

[<oechem.OEMolBase; proxy of <Swig Object of type 'OEChem::OEMolBase *' at 0x7f0ef9b597b0> >]

Next will be visualization with NGLView and calculation of RMSD between predicted and experimental observed binding pose ...